In [30]:
import pandas as pd
import regex as re

In [2]:
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)

# Reading in the csv as dataframes
naming conventions chosen by Matthew

In [3]:
onboard_df = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')
onboard_df.head()
#in long form, may need to pivot remember the data is organized. faultid=recordid?

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [76]:
fault_df = pd.read_csv('../data/J1939Faults.csv')
fault_df.head(20)

C:\Users\zylst\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
6,7,990439,2015-02-21 11:40:52.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29.000
7,8,990441,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000
8,9,990442,2015-02-21 11:40:22.000,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,15,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000
9,10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1417,105438630,33.039953,-96.182592,2015-02-21 11:41:51.000


# Cleaning fault data

##### From the readme: 
<br> "Remove faults occurring in the vicinity of the service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)" <br> <br>


We want to go with 4 decimal points to get within 11.1m.

In [77]:
fault_df[['Latitude', 'Longitude']] = fault_df[['Latitude','Longitude']].astype(str)

In [78]:
fault_df['Latitude'] = fault_df['Latitude'].str.extract(r'(\-?\d+.\d{4})')

In [79]:
fault_df['Longitude'] = fault_df['Longitude'].str.extract(r'(\-?\d+.\d{4})')

In [83]:
#fault_df.loc[fault_df.Latitude == 35.5883]
#fault_df.loc[fault_df.Latitude == 36.0666]
#fault_df.loc[fault_df.Latitude == 36.1950]
fault_df.shape

(1187335, 20)

In [85]:
fault_df[['Latitude', 'Longitude']] = fault_df[['Latitude','Longitude']].astype(float)

In [86]:
lat_only = fault_df.loc[(fault_df.Latitude == 35.5883) | (fault_df.Latitude == 36.0666) | (fault_df.Latitude == 36.1950)]
lat_only.shape

(79225, 20)

In [88]:
lat_long = fault_df.loc[(fault_df.Latitude == 36.1950) & (fault_df.Longitude == -83.174)]
lat_long.shape

(7265, 20)

In [ ]:
#We want to get close, not this exact lat/long, will need to look at maeva's table for lat/long closeness plus how many digits it goes to

In [19]:
#fault_df[fault_df['Latitude'] == 36.1950 & fault_df['Longitude'] ==  -83.174722]

fault_df = fault_df.loc[((fault_df.Latitude != 36.1950) | (fault_df.Longitude != -83.174722))]

#fault_df = fault_df.loc[((fault_df.Latitude != 36.1950) & (fault_df.Longitude != -83.174722))]

#both give the same answer however it doesn't make a ton of sense that only 75 exist, yet over 3,000 are dropped

In [20]:
fault_df.shape

(1187260, 20)

##### From the readme:
<br> "Remove faults where the EquipmentID has more than 5 characters."

In [10]:
#confirmed that NaN values are not necessary
fault_df['EquipmentID'] = fault_df['EquipmentID'].dropna()

In [11]:
fault_df = fault_df[fault_df['EquipmentID'].str.len()<=5] 

In [12]:
fault_df.shape

(64977, 20)

In [13]:
fault_df['EquipmentID'].str.len().unique()

array([4, 3, 5], dtype=int64)

##### our own analysis:
<br> actionDescription,faultValue,ecuSource were all empty columns that we'll remove.

In [14]:
fault_df = fault_df.drop(['actionDescription','faultValue','ecuSource'],axis=1)

In [15]:
fault_df.shape

(64977, 17)